In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
print("Libraries imported successfully! ✅")

Libraries imported successfully! ✅


In [2]:
df = pd.read_csv('spotify-2023.csv', encoding='latin-1')

# Fix encoding issues in text columns
import re

def clean_text(text):
    if isinstance(text, str):
        try:
            text = text.encode('latin-1').decode('utf-8')
        except:
            pass
        # Remove non-printable characters
        text = re.sub(r'[^\x20-\x7E]', '', text)
        # Remove garbage repeated characters like yyyyyyy
        text = re.sub(r'(.)\1{4,}', '', text)
        text = text.strip()
    return text

df['track_name'] = df['track_name'].apply(clean_text)
df['artist(s)_name'] = df['artist(s)_name'].apply(clean_text)

print(f"Dataset Shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
df.head()

Dataset Shape: (953, 24)

Columns: ['track_name', 'artist(s)_name', 'artist_count', 'released_year', 'released_month', 'released_day', 'in_spotify_playlists', 'in_spotify_charts', 'streams', 'in_apple_playlists', 'in_apple_charts', 'in_deezer_playlists', 'in_deezer_charts', 'in_shazam_charts', 'bpm', 'key', 'mode', 'danceability_%', 'valence_%', 'energy_%', 'acousticness_%', 'instrumentalness_%', 'liveness_%', 'speechiness_%']


,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703,43,...,125,B,Major,80,89,83,31,0,8,4
1,LALA,Myke Towers,1,2023,3,23,1474,48,133716286,48,...,92,C#,Major,71,61,74,7,0,10,4
2,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974,94,...,138,F,Major,51,32,53,17,0,31,6
3,Cruel Summer,Taylor Swift,1,2019,8,23,7858,100,800840817,116,...,170,A,Major,55,58,72,11,0,11,15
4,WHERE SHE GOES,Bad Bunny,1,2023,5,18,3133,50,303236322,84,...,144,A,Minor,65,23,80,14,63,11,6


In [3]:
print("Null Values:\n", df.isnull().sum())

# Only drop rows where CRITICAL columns are null
df = df.dropna(subset=['track_name', 'artist(s)_name', 'streams'])

# Fix streams to numeric
df['streams'] = pd.to_numeric(df['streams'], errors='coerce')
df = df.dropna(subset=['streams'])

# Remove streams = 0
df = df[df['streams'] > 0]

# Strip column names
df.columns = df.columns.str.strip()

# Fill non-critical nulls with Unknown or median
df['key'] = df['key'].fillna('Unknown')
df['in_shazam_charts'] = pd.to_numeric(df['in_shazam_charts'], errors='coerce').fillna(0)

# Popularity category
df['popularity_category'] = pd.qcut(df['streams'], q=3,
                                     labels=['Low', 'Medium', 'High'])

print(f"\nCleaned Data Shape: {df.shape}")
print(f"Remaining Nulls:\n{df.isnull().sum()[df.isnull().sum() > 0]}")
df.head()

Null Values:
 track_name               0
artist(s)_name           0
artist_count             0
released_year            0
released_month           0
released_day             0
in_spotify_playlists     0
in_spotify_charts        0
streams                  0
in_apple_playlists       0
in_apple_charts          0
in_deezer_playlists      0
in_deezer_charts         0
in_shazam_charts        50
bpm                      0
key                     95
mode                     0
danceability_%           0
valence_%                0
energy_%                 0
acousticness_%           0
instrumentalness_%       0
liveness_%               0
speechiness_%            0
dtype: int64

Cleaned Data Shape: (952, 25)
Remaining Nulls:
Series([], dtype: int64)


,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%,popularity_category
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703.0,43,...,B,Major,80,89,83,31,0,8,4,Low
1,LALA,Myke Towers,1,2023,3,23,1474,48,133716286.0,48,...,C#,Major,71,61,74,7,0,10,4,Low
2,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974.0,94,...,F,Major,51,32,53,17,0,31,6,Low
3,Cruel Summer,Taylor Swift,1,2019,8,23,7858,100,800840817.0,116,...,A,Major,55,58,72,11,0,11,15,High
4,WHERE SHE GOES,Bad Bunny,1,2023,5,18,3133,50,303236322.0,84,...,A,Minor,65,23,80,14,63,11,6,Medium


In [4]:
print("📊 Basic Statistics:")
print(f"Total Songs: {len(df)}")
print(f"Total Artists: {df['artist(s)_name'].nunique()}")
print(f"Year Range: {df['released_year'].min()} - {df['released_year'].max()}")
print(f"Most Streamed Song: {df.loc[df['streams'].idxmax(), 'track_name']}")
print(f"Max Streams: {df['streams'].max():,.0f}")
print(f"\nTop 5 Most Streamed Songs:")
print(df[['track_name', 'artist(s)_name', 'streams']]
      .sort_values('streams', ascending=False)
      .head())

📊 Basic Statistics:
Total Songs: 952
Total Artists: 644
Year Range: 1930 - 2023
Most Streamed Song: Blinding Lights
Max Streams: 3,703,895,074

Top 5 Most Streamed Songs:
                                        track_name         artist(s)_name  \
55                                 Blinding Lights             The Weeknd   
179                                   Shape of You             Ed Sheeran   
86                               Someone You Loved          Lewis Capaldi   
620                                   Dance Monkey            Tones and I   
41   Sunflower - Spider-Man: Into the Spider-Verse  Post Malone, Swae Lee   

          streams  
55   3.703895e+09  
179  3.562544e+09  
86   2.887242e+09  
620  2.864792e+09  
41   2.808097e+09  


In [5]:
df.to_csv('spotify_updatedd.csv', index=False)
print("Saved!")

Saved!


In [6]:
# Always check unique values in important text columns
print(df['track_name'].unique())
print(df['artist(s)_name'].unique())

['Seven (feat. Latto) (Explicit Ver.)' 'LALA' 'vampire' 'Cruel Summer'
 'WHERE SHE GOES' 'Sprinter' 'Ella Baila Sola' 'Columbia' 'fukumean'
 'La Bebe - Remix' 'un x100to' 'Super Shy' 'Flowers' 'Daylight'
 'As It Was' 'Kill Bill' 'Cupid - Twin Ver.'
 'What Was I Made For? [From The Motion Picture "Barbie"]' 'Classy 101'
 'Like Crazy' 'LADY GAGA' 'I Can See You (Taylors Version) (From The'
 'I Wanna Be Yours' 'Peso Pluma: Bzrp Music Sessions, Vol. 55'
 'Popular (with Playboi Carti & Madonna) - The Idol Vol. 1 (Music from the HBO Original Series)'
 'SABOR FRESA' 'Calm Down (with Selena Gomez)' 'MOJABI GHOST' 'Last Night'
 'Dance The Night (From Barbie The Album)' 'Rush' 'TULUM' "Creepin'"
 'Anti-Hero' 'TQG' 'Los del Espacio' 'Frgil (feat. Grupo Front'
 'Blank Space' 'Style' 'TQM' 'El Azul'
 'Sunflower - Spider-Man: Into the Spider-Verse' "I'm Good (Blue)"
 'See You Again' 'Barbie World (with Aqua) [From Barbie The Album]'
 'Angels Like You' "I Ain't Worried" 'Die For You' 'Starboy'
 'Die 

In [7]:
# Fix 1 - Drop 2 rows where track_name is null
df = df.dropna(subset=['track_name'])
print(f"Removed null track names. Rows remaining: {len(df)}")

# Fix 2 - Fix in_deezer_playlists (has commas like 1,000)
df['in_deezer_playlists'] = df['in_deezer_playlists'].astype(str).str.replace(',', '').str.strip()
df['in_deezer_playlists'] = pd.to_numeric(df['in_deezer_playlists'], errors='coerce').fillna(0)
print(f"Fixed in_deezer_playlists column")

# Fix 3 - Convert streams to integer
df['streams'] = df['streams'].astype(int)
print(f"Streams converted to integer")

# Final check
print(f"\n Final Data Summary:")
print(f"   Total Songs    : {len(df)}")
print(f"   Total Columns  : {len(df.columns)}")
print(f"   Null Values    : {df.isnull().sum().sum()}")
print(f"   Streams Type   : {df['streams'].dtype}")
print(f"   Deezer Type    : {df['in_deezer_playlists'].dtype}")


Removed null track names. Rows remaining: 952
Fixed in_deezer_playlists column
Streams converted to integer

 Final Data Summary:
   Total Songs    : 952
   Total Columns  : 25
   Null Values    : 0
   Streams Type   : int32
   Deezer Type    : int64


In [8]:
# Fix key column - replace Unknown with Not Available
df['key'] = df['key'].replace('Unknown', 'Not Available')

# Verify
print("Key column values:")
print(df['key'].value_counts())

Key column values:
key
C#               120
G                 96
Not Available     95
G#                91
F                 89
B                 81
D                 81
A                 74
F#                73
E                 62
A#                57
D#                33
Name: count, dtype: int64


In [9]:
df.to_csv('spotify_final.csv', index=False)
print(f"\n✅ Final clean file saved as spotify_final.csv")


✅ Final clean file saved as spotify_final.csv
